In [ ]:
import numpy as np
import scipy.signal as signal
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from IPython.display import Audio, display
import sys

def main():
    print("1> Manual upload.\n2> Sine wave.\n3> Cosine wave.")
    n=int(input("Please choose your message signal type:"))
    if (n==1):
        data, fs = upload()
        length = len(data)
    elif ((n==2) or (n==3)):
        fs = int(input("Enter frequency of message signal:"))
        duration = int(input("Duration of the signal:"))
        sr = int(input("Enter sampling rate:"))
        data = sinu(n,fs,duration,sr)
        length = len(data)
    else:
        print("Invalid choice!")
    carrier_freq = int(input("Enter carrier frequency:"))
    xc = carrier(fs, length, carrier_freq)

    print("Type of modulation:\n\t1> DSB\n\t2> AM\n\t3> SSB\n\t4> VSB\n\t5> Phase Modulation\n\t6> FM")
    mtype = int(input("Enter choice:"))

    if (mtype==1):
        modulated = modulate_dsb_sc(data, fs, carrier_freq)
        title = "DSB-SC"
        demodulated = demodulate_coherent(modulatedDSB, fs, carrier_freq)
        envelope = envelope_detection(modulatedDSB)
        
    elif (mtype==2):
        modulated = modulate_am(data, fs, carrier_freq)
        title = "AM"
        demodulated = demodulate_coherent(modulatedAM, fs, carrier_freq)
        envelope = envelope_detection(modulatedAM)
        
    elif (mtype==3):
        modulated = modulate_ssb(data, fs, carrier_freq)
        title = "SSB"
        demodulated = demodulate_coherent(modulatedSSB, fs, carrier_freq)
        envelope = envelope_detection(modulatedSSB)
        
    elif (mtype==4):
        modulated = modulate_vsb(data, fs, carrier_freq)
        title = "VSB"
        demodulated = demodulate_coherent(modulatedVSB, fs, carrier_freq)
        envelope = envelope_detection(modulatedVSB)

    elif (mtype==5):
        modulated = phase_modulation(data, fs, carrier_freq)
        title = "PM"
        pll1 = pll_track(modulated, Kp=50, Ki=0, pll_type=1)
        pll2 = pll_track(modulated, Kp=30, Ki=10, pll_type=2)
        pll3 = pll_track(modulated, Kp=30, Ki=10, Kd=5, pll_type=3)
        
    elif (mtype==6):
        modulated = frequency_modulation(data, fs, carrier_freq)
        title = "FM"
        pll1 = pll_track(modulated, Kp=50, Ki=0, pll_type=1)
        pll2 = pll_track(modulated, Kp=30, Ki=10, pll_type=2)
        pll3 = pll_track(modulated, Kp=30, Ki=10, Kd=5, pll_type=3)
    else:
        print("Invalid choice!")
        sys.exit(1)
        
    play_audio(data, fs, "Original Audio")
    plot_time(data, fs, "Original")
    plot_spectrum(data, fs, "Original")
    plot_time(modulated, fs, "Modulated")
    plot_spectrum(modulated, fs, "Modulated")
    
    if((mtype!=5) or (mtype!=6)):
        play_audio(demodulated, fs, "Coherent Demodulated Audio")
        play_audio(envelope, fs, "Envelope Detection Demodulated Audio")
        plot_time(demodulated, fs, "Demodulated")
        plot_spectrum(demodulated, fs, "Coherent Demodulated Audio")
        plot_time(envelope, fs, "Envelope Detection Demodulated Audio")
        plot_spectrum(envelope, fs, "Envelope Detection Demodulated Audio")
    else:
        play_audio(demodulated, fs, "PLL Demodulated Audio")
        play_audio(envelope, fs, "Demodulated Audio using time domain approach")
        plot_time(demodulated, fs, "PLL Demodulated Audio")
        plot_spectrum(demodulated, fs, "PLL Demodulated Audio")
        plot_time(envelope, fs, "Demodulated Audio using time domain approach")
        plot_spectrum(envelope, fs, "Demodulated Audio using time domain approach")

if __name__ == "__main__":
    main()

In [ ]:
#message signal

def upload():
    s = input("Enter file path:")
    start_time = int(input("Enter start time:"))
    duration = int(input("Enter duration:"))
    filename = s
    fs, data = wav.read(filename)
    if data.ndim > 1:
        data = np.mean(data, axis=1)
    start_sample = int(start_time * fs)
    end_sample = int((start_time + duration) * fs)
    segment_data = data[start_sample:end_sample]
    normalized_data = segment_data / np.max(np.abs(segment_data))
    return normalized_dat, fs

def sinu(n, fm, duration, sampling_rate):
    t = np.linspace(0, duration, int(sampling_rate * duration), endpoint=False)
    if(n==2):
        y = np.sin(2*np.pi*fm*t)
    else:
        y = np.cos(2*np.pi*fm*t)
    return y

In [ ]:
#carrier generation

def carrier(fs, length, fc):
    t = np.arange(length) / fs
    return np.cos(2 * np.pi * carrier_freq * t)

In [ ]:
#Amplitude modulation

def modulate_dsb_sc(data, fs, carrier_freq):
    carrier = generate_carrier(fs, len(data), carrier_freq)
    return data * carrier

def modulate_ssb(data, fs, carrier_freq):
    hilbert_data = signal.hilbert(data)
    carrier = generate_carrier(fs, len(data), carrier_freq)
    ssb_modulated = np.real(hilbert_data) * carrier - np.imag(hilbert_data) * np.sin(2 * np.pi * carrier_freq * np.arange(len(data)) / fs)
    return ssb_modulated

def modulate_am(data, fs, carrier_freq):
    carrier = generate_carrier(fs, len(data), carrier_freq)
    return (1 + data) * carrier

def modulate_vsb(data, fs, carrier_freq):
    ssb_signal = modulate_ssb(data, fs, carrier_freq)
    vsb_filter = signal.firwin(101, [4000/(fs/2), 6000/(fs/2)], pass_zero=False)
    vsb_modulated = signal.lfilter(vsb_filter, 1.0, ssb_signal)
    return vsb_modulated

In [ ]:
#Amplitude demodulation

def demodulate_coherent(modulated, fs, carrier_freq):
    carrier = generate_carrier(fs, len(modulated), carrier_freq)
    demodulated = modulated * carrier
    demodulated = signal.lfilter(*signal.butter(4, 3000/(fs/2), 'low'), demodulated)
    return demodulated

def envelope_detection(modulated):
    return np.abs(signal.hilbert(modulated))

In [ ]:
#Frequency modulation

def frequency_modulation(signal, fs, fc):
    time = np.linspace(0, len(signal) / fs, num=len(signal))
    integrated_signal = np.cumsum(signal) / fs
    t = np.arange(len(signal)) / fs
    fm_signal = np.cos(2*np.pi*fc*t + integrated_signal)
    return fm_signal

def phase_modulation(signal, fs, fc):
    time = np.linspace(0, len(signal) / fs, num=len(signal))
    t = np.arange(len(signal)) / fs
    fm_signal = np.cos(2*np.pi*fc*t + signal)
    return fm_signal    

In [ ]:
#Frequency demodulation using PLL

def pll_track(fm_signal, Kp, Ki, Kd=0, pll_type=1, dt=0.0001, integrator_limit=500, derivative_limit=50):
    phase_error = 0
    phase = 0
    freq_est = np.zeros_like(fm_signal)
    integrator = 0
    differentiator = 0
    prev_error = 0
    
    for i in range(1, len(fm_signal)):
        phase_error = fm_signal[i] - phase
        
        if pll_type == 1:
            control = Kp * phase_error
        elif pll_type == 2:
            integrator += Ki * phase_error * dt
            integrator = np.clip(integrator, -integrator_limit, integrator_limit)
            control = Kp * phase_error + integrator
        elif pll_type == 3:
            integrator += Ki * phase_error * dt
            integrator = np.clip(integrator, -integrator_limit, integrator_limit)
            differentiator = Kd * (phase_error - prev_error) / dt
            differentiator = np.clip(differentiator, -derivative_limit, derivative_limit)
            control = Kp * phase_error + integrator + differentiator
        
        phase += control * dt
        freq_est[i] = control
        prev_error = phase_error
    
    return freq_est

In [ ]:
"""import control as ctrl 
s = ctrl.TransferFunction.s
Kp = 50
H1 = Kp / s
Ki = 10
H2 = (Kp * s + Ki) / (s**2)
Kd = 5
H3 = (Kp * s**2 + Ki * s + Kd) / (s**3)

timefm, response1 = ctrl.forced_response(H1 / s, T=np.linspace(0, 1.5, 1500))
timefm, response2 = ctrl.forced_response(H2 / s, T=np.linspace(0, 1.5, 1500))
timefm, response3 = ctrl.forced_response(H3 / s, T=np.linspace(0, 1.5, 1500))"""

In [ ]:
#play audio

def play_audio(signal, fs, title="Audio"):
    print(f"\nPlaying {title}...")
    display(Audio(signal, rate=fs))

In [ ]:
#Plotting signal

def plot_spectrum(signal, fs, title):
    N = len(signal)
    freq = np.linspace(-fs/2, fs/2, N)
    spectrum = np.abs(fft.fftshift(fft.fft(signal)))

    plt.figure(figsize=(10, 4))
    plt.plot(freq, spectrum)
    plt.title(title)
    plt.xlabel("Frequency (Hz)")
    plt.ylabel("Magnitude")
    plt.grid()
    plt.show()

def plot_time(signal, fs, title):
    time = np.linspace(0, len(signal) / fs, num=len(signal))
    plt.figure(figsize=(12, 5))
    plt.subplot(2, 1, 1)
    plt.plot(time, signal, label="Original Signal", color='b')
    plt.xlabel("Time (seconds)")
    plt.ylabel("Amplitude")
    plt.title("Original Audio Signal")
    plt.legend()